# Production SQL Query Practice

This notebook contains common SQL queries and patterns frequently used in production environments. It covers everything from basic queries to advanced optimization techniques, with practical examples and explanations.

## Table of Contents
1. Database Connection Setup
2. Schema and Sample Data Creation
3. Basic SELECT Operations
4. JOIN Operations
5. Aggregation and GROUP BY
6. Window Functions
7. Common Table Expressions (CTEs)
8. Subqueries
9. Indexing and Query Optimization
10. Transactions and Concurrency
11. Data Manipulation (DML)
12. Upsert Patterns
13. Performance Best Practices
14. Security Considerations
15. Data Import/Export
16. VS Code Integration

Let's start by setting up our database connection and required libraries.

# 1. Database Connection Setup

First, let's install and import the required libraries. We'll use SQLite for this practice as it doesn't require a separate server, but the concepts apply to any SQL database.

Note: For production environments, you would typically use PostgreSQL, MySQL, or other enterprise databases.

In [ ]:
# Import required libraries
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Helper function to execute SQL queries and return results as DataFrame
def run_query(query, params=None, connection=None):
    """
    Execute a SQL query and return results as a pandas DataFrame
    
    Parameters:
    - query: SQL query string
    - params: Optional parameters for parameterized queries
    - connection: Optional SQLite connection (will create new one if not provided)
    
    Returns:
    - pandas DataFrame with query results
    """
    if connection is None:
        connection = sqlite3.connect('practice_db.sqlite')
    
    try:
        if params:
            result = pd.read_sql_query(query, connection, params=params)
        else:
            result = pd.read_sql_query(query, connection)
        return result
    except Exception as e:
        print(f"Error executing query: {e}")
        return None
    finally:
        if connection is None:  # Only close if we created it
            connection.close()

# Create a connection to our practice database
conn = sqlite3.connect('practice_db.sqlite')
print("Database connection established successfully!")

# 2. Creating Schema and Sample Data

Let's create a realistic e-commerce database schema with the following tables:
- users: Customer information
- products: Product catalog
- orders: Order header information
- order_items: Order line items
- categories: Product categories

We'll add proper constraints, indexes, and relationships between these tables.

In [ ]:
# Create tables with proper constraints and indexes
create_tables_sql = """
-- Drop existing tables if they exist
DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS categories;
DROP TABLE IF EXISTS users;

-- Create users table
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    email TEXT UNIQUE NOT NULL,
    name TEXT NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status TEXT CHECK (status IN ('active', 'inactive', 'suspended')) DEFAULT 'active'
);
CREATE INDEX idx_users_email ON users(email);

-- Create categories table
CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT,
    parent_category_id INTEGER,
    FOREIGN KEY (parent_category_id) REFERENCES categories(category_id)
);
CREATE INDEX idx_categories_parent ON categories(parent_category_id);

-- Create products table
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT,
    price DECIMAL(10,2) NOT NULL CHECK (price >= 0),
    category_id INTEGER,
    stock_quantity INTEGER DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
);
CREATE INDEX idx_products_category ON products(category_id);
CREATE INDEX idx_products_price ON products(price);

-- Create orders table
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    status TEXT CHECK (status IN ('pending', 'processing', 'shipped', 'delivered', 'cancelled')),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    total_amount DECIMAL(10,2) DEFAULT 0,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
CREATE INDEX idx_orders_user ON orders(user_id);
CREATE INDEX idx_orders_created_at ON orders(created_at);

-- Create order_items table
CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL CHECK (quantity > 0),
    price_at_time DECIMAL(10,2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
CREATE INDEX idx_order_items_order ON order_items(order_id);
CREATE INDEX idx_order_items_product ON order_items(product_id);
"""

# Execute the SQL to create tables
conn.executescript(create_tables_sql)
print("Tables created successfully!")

# Insert sample data
def insert_sample_data():
    # Insert categories
    categories_data = [
        (1, 'Electronics', 'Electronic devices and accessories', None),
        (2, 'Clothing', 'Apparel and accessories', None),
        (3, 'Books', 'Books and publications', None),
        (4, 'Smartphones', 'Mobile phones and accessories', 1),
        (5, 'Laptops', 'Portable computers', 1)
    ]
    conn.executemany("INSERT INTO categories (category_id, name, description, parent_category_id) VALUES (?, ?, ?, ?)",
                     categories_data)
    
    # Insert users
    users_data = [
        (1, 'john.doe@email.com', 'John Doe', 'active'),
        (2, 'jane.smith@email.com', 'Jane Smith', 'active'),
        (3, 'bob.wilson@email.com', 'Bob Wilson', 'inactive')
    ]
    conn.executemany("INSERT INTO users (user_id, email, name, status) VALUES (?, ?, ?, ?)",
                     users_data)
    
    # Insert products
    products_data = [
        (1, 'iPhone 13', 'Latest iPhone model', 999.99, 4, 50),
        (2, 'MacBook Pro', 'Professional laptop', 1299.99, 5, 30),
        (3, 'Python Programming', 'Programming book', 49.99, 3, 100),
        (4, 'T-shirt', 'Cotton t-shirt', 19.99, 2, 200)
    ]
    conn.executemany("INSERT INTO products (product_id, name, description, price, category_id, stock_quantity) VALUES (?, ?, ?, ?, ?, ?)",
                     products_data)
    
    # Insert orders and order items
    orders_data = [
        (1, 1, 'delivered', '2023-01-01', 1019.98),
        (2, 2, 'processing', '2023-01-02', 1299.99),
        (3, 1, 'pending', '2023-01-03', 69.98)
    ]
    conn.executemany("INSERT INTO orders (order_id, user_id, status, created_at, total_amount) VALUES (?, ?, ?, ?, ?)",
                     orders_data)
    
    order_items_data = [
        (1, 1, 1, 1, 999.99),  # iPhone in order 1
        (2, 1, 4, 1, 19.99),   # T-shirt in order 1
        (3, 2, 2, 1, 1299.99), # MacBook in order 2
        (4, 3, 4, 2, 19.99),   # T-shirt in order 3
        (5, 3, 3, 1, 49.99)    # Book in order 3
    ]
    conn.executemany("INSERT INTO order_items (order_item_id, order_id, product_id, quantity, price_at_time) VALUES (?, ?, ?, ?, ?)",
                     order_items_data)
    
    conn.commit()
    print("Sample data inserted successfully!")

# Insert the sample data
insert_sample_data()

# 3. Basic SELECT Operations

Let's start with some common SELECT operations used in production. We'll cover:
- Basic SELECT with filtering
- Using WHERE clauses effectively
- ORDER BY and LIMIT
- DISTINCT values
- Pattern matching with LIKE
- Working with dates

In [ ]:
# Basic SELECT with filtering
query = """
SELECT 
    product_id,
    name,
    price,
    stock_quantity
FROM products
WHERE price > 100
ORDER BY price DESC;
"""
run_query(query)

# Using DISTINCT to find unique order statuses
query = """
SELECT DISTINCT status
FROM orders;
"""
run_query(query)

# Pattern matching with LIKE
query = """
SELECT 
    name,
    description,
    price
FROM products
WHERE name LIKE '%Book%'
   OR description LIKE '%book%';
"""
run_query(query)

# Date filtering and formatting
query = """
SELECT 
    order_id,
    user_id,
    date(created_at) as order_date,
    status,
    total_amount
FROM orders
WHERE date(created_at) >= date('2023-01-02')
ORDER BY created_at DESC;
"""
run_query(query)

# Complex filtering with multiple conditions
query = """
SELECT 
    p.name,
    p.price,
    p.stock_quantity,
    c.name as category
FROM products p
JOIN categories c ON p.category_id = c.category_id
WHERE p.price BETWEEN 10 AND 1000
    AND p.stock_quantity > 0
    AND c.parent_category_id IS NOT NULL
ORDER BY p.price;
"""
run_query(query)

# 4. JOIN Operations

Now let's look at different types of JOINs commonly used in production:
- INNER JOIN
- LEFT JOIN
- Multiple table joins
- Self joins (for hierarchical data)
- Anti-joins (finding missing relationships)

In [ ]:
# INNER JOIN - Find all orders with their products
query = """
SELECT 
    o.order_id,
    u.name as customer_name,
    p.name as product_name,
    oi.quantity,
    oi.price_at_time,
    o.status
FROM orders o
INNER JOIN order_items oi ON o.order_id = oi.order_id
INNER JOIN products p ON oi.product_id = p.product_id
INNER JOIN users u ON o.user_id = u.user_id
ORDER BY o.order_id;
"""
run_query(query)

# LEFT JOIN - Find all products and their orders (including products never ordered)
query = """
SELECT 
    p.product_id,
    p.name,
    COUNT(oi.order_item_id) as times_ordered,
    COALESCE(SUM(oi.quantity), 0) as total_quantity_ordered
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.name
ORDER BY times_ordered DESC;
"""
run_query(query)

# Self JOIN - Find all categories with their parent category names
query = """
SELECT 
    c1.category_id,
    c1.name as category_name,
    c2.name as parent_category_name
FROM categories c1
LEFT JOIN categories c2 ON c1.parent_category_id = c2.category_id
ORDER BY c2.name, c1.name;
"""
run_query(query)

# Anti-join - Find products that have never been ordered
query = """
SELECT 
    p.product_id,
    p.name,
    p.price,
    p.stock_quantity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
WHERE oi.order_item_id IS NULL;
"""
run_query(query)

# Complex multiple joins with conditions
query = """
SELECT 
    c.name as category_name,
    p.name as product_name,
    COUNT(DISTINCT o.order_id) as number_of_orders,
    SUM(oi.quantity) as total_units_sold,
    SUM(oi.quantity * oi.price_at_time) as total_revenue
FROM categories c
JOIN products p ON c.category_id = p.category_id
LEFT JOIN order_items oi ON p.product_id = oi.product_id
LEFT JOIN orders o ON oi.order_id = o.order_id
WHERE o.status != 'cancelled' OR o.status IS NULL
GROUP BY c.category_id, c.name, p.product_id, p.name
HAVING total_units_sold > 0
ORDER BY total_revenue DESC;
"""
run_query(query)

# 5. Aggregation with GROUP BY

Let's explore common aggregation patterns:
- Basic GROUP BY with COUNT, SUM, AVG
- Multiple grouping levels
- HAVING clause
- Complex calculations
- Combining aggregations with joins

In [ ]:
# Basic aggregation - Sales by category
query = """
SELECT 
    c.name as category_name,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(oi.quantity) as total_units_sold,
    ROUND(AVG(oi.price_at_time), 2) as avg_price,
    ROUND(SUM(oi.quantity * oi.price_at_time), 2) as total_revenue
FROM categories c
JOIN products p ON c.category_id = p.category_id
JOIN order_items oi ON p.product_id = oi.product_id
JOIN orders o ON oi.order_id = o.order_id
GROUP BY c.category_id, c.name
ORDER BY total_revenue DESC;
"""
run_query(query)

# Multiple grouping levels - Sales by category and status
query = """
SELECT 
    c.name as category_name,
    o.status,
    COUNT(DISTINCT o.order_id) as num_orders,
    ROUND(SUM(oi.quantity * oi.price_at_time), 2) as total_revenue
FROM categories c
JOIN products p ON c.category_id = p.category_id
JOIN order_items oi ON p.product_id = oi.product_id
JOIN orders o ON oi.order_id = o.order_id
GROUP BY c.category_id, c.name, o.status
ORDER BY c.name, total_revenue DESC;
"""
run_query(query)

# Using HAVING - Find popular products (ordered more than once)
query = """
SELECT 
    p.name as product_name,
    COUNT(DISTINCT o.order_id) as num_orders,
    SUM(oi.quantity) as total_quantity,
    ROUND(AVG(oi.price_at_time), 2) as avg_price,
    ROUND(SUM(oi.quantity * oi.price_at_time), 2) as total_revenue
FROM products p
JOIN order_items oi ON p.product_id = oi.product_id
JOIN orders o ON oi.order_id = o.order_id
GROUP BY p.product_id, p.name
HAVING num_orders > 1
ORDER BY total_revenue DESC;
"""
run_query(query)

# Complex calculations - Customer purchase patterns
query = """
SELECT 
    u.name as customer_name,
    COUNT(DISTINCT o.order_id) as total_orders,
    ROUND(AVG(o.total_amount), 2) as avg_order_value,
    ROUND(MAX(o.total_amount), 2) as largest_order,
    ROUND(MIN(o.total_amount), 2) as smallest_order,
    ROUND(SUM(o.total_amount), 2) as total_spent,
    ROUND(SUM(o.total_amount) / COUNT(DISTINCT o.order_id), 2) as avg_order_size
FROM users u
LEFT JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
ORDER BY total_spent DESC;
"""
run_query(query)

# 6. Window Functions

Window functions are extremely powerful for analytics and reporting. Let's look at:
- ROW_NUMBER(), RANK(), DENSE_RANK()
- Running totals and moving averages
- Partitioning data
- Lead and lag functions

In [ ]:
# Ranking products by price within categories
query = """
SELECT 
    c.name as category_name,
    p.name as product_name,
    p.price,
    ROW_NUMBER() OVER (PARTITION BY c.category_id ORDER BY p.price DESC) as price_rank,
    RANK() OVER (PARTITION BY c.category_id ORDER BY p.price DESC) as price_rank_with_ties,
    DENSE_RANK() OVER (PARTITION BY c.category_id ORDER BY p.price DESC) as dense_price_rank
FROM products p
JOIN categories c ON p.category_id = c.category_id
ORDER BY c.name, p.price DESC;
"""
run_query(query)

# Running totals and moving averages
query = """
SELECT 
    o.order_id,
    o.created_at,
    o.total_amount,
    SUM(o.total_amount) OVER (ORDER BY o.created_at) as running_total,
    AVG(o.total_amount) OVER (
        ORDER BY o.created_at 
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) as moving_avg_3_orders
FROM orders o
ORDER BY o.created_at;
"""
run_query(query)

# Customer purchase rankings and percentages
query = """
SELECT 
    u.name as customer_name,
    COUNT(o.order_id) as total_orders,
    ROUND(SUM(o.total_amount), 2) as total_spent,
    RANK() OVER (ORDER BY SUM(o.total_amount) DESC) as spending_rank,
    ROUND(
        SUM(o.total_amount) * 100.0 / SUM(SUM(o.total_amount)) OVER (), 
        2
    ) as percentage_of_total_sales
FROM users u
LEFT JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
ORDER BY total_spent DESC;
"""
run_query(query)

# Lead/Lag to analyze order patterns
query = """
SELECT 
    u.name as customer_name,
    o.order_id,
    o.created_at,
    o.total_amount,
    LAG(o.total_amount) OVER (
        PARTITION BY u.user_id 
        ORDER BY o.created_at
    ) as previous_order_amount,
    LEAD(o.total_amount) OVER (
        PARTITION BY u.user_id 
        ORDER BY o.created_at
    ) as next_order_amount,
    ROUND(
        o.total_amount - LAG(o.total_amount) OVER (
            PARTITION BY u.user_id 
            ORDER BY o.created_at
        ),
        2
    ) as amount_change_from_previous
FROM orders o
JOIN users u ON o.user_id = u.user_id
ORDER BY u.name, o.created_at;
"""
run_query(query)